# Modelling NIQS Hardware TF

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *

In [2]:
def apply_map(state, kraus_list):
    state = [K@state@tf.linalg.adjoint(K) for K in kraus_list]
    state = tf.math.reduce_sum(tf.stack(state), axis=0)
    return state


def expectation_value(state, observable):
    ev = tf.linalg.trace(observable@state)
    return ev

In [9]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 8

A = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
B = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
G = A + 1j*B
Q, R = tf.linalg.qr(G, full_matrices = False)
D = tf.linalg.tensor_diag_part(R)
D = tf.math.sign(D)
D = tf.linalg.diag(D)
U = Q@D


kraus_target_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]

In [10]:
N = 1000
state_index, observ_index = index_generator(n, N, trace=False)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = prepare_input(config)

    config = numberToBase(j, 3, n)
    observable = pauli_observable(config)
    
    input_list.append([state, observable])

target_list = [expectation_value(apply_map(input[0], kraus_target_list), input[1]) for input in input_list]
optimizer = tf.optimizers.Adam(learning_rate=0.05)

model = 

In [11]:
for i in range(1000):
    with tf.GradientTape() as tape:
        G = A + 1j*B
        Q, R = tf.linalg.qr(G, full_matrices = False)
        D = tf.linalg.tensor_diag_part(R)
        D = tf.math.sign(D)
        D = tf.linalg.diag(D)
        U = Q@D

        kraus_model_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]
        pred_list = [expectation_value(apply_map(input[0], kraus_model_list), input[1]) for input in input_list]
        loss = tf.math.reduce_mean(tf.stack([(target - predicted)**2 for target, predicted in zip(target_list, pred_list)]))

    grads = tape.gradient(loss, [A, B])
    optimizer.apply_gradients(zip(grads, [A, B]))
    
    print(f"loss: {loss: .4f}, {tf.math.reduce_mean(tf.math.abs(G))}")

loss:  0.0292+0.0000j, 1.301522135734558
loss:  0.0258+0.0000j, 1.3030211925506592
loss:  0.0229+0.0000j, 1.3061857223510742
loss:  0.0203+0.0000j, 1.3110737800598145
loss:  0.0181+0.0000j, 1.3191564083099365
loss:  0.0163+0.0000j, 1.328444004058838
loss:  0.0148+0.0000j, 1.3391997814178467
loss:  0.0136+0.0000j, 1.3502404689788818
loss:  0.0125+0.0000j, 1.3619322776794434
loss:  0.0115+0.0000j, 1.3748447895050049
loss:  0.0106+0.0000j, 1.3881510496139526
loss:  0.0098+0.0000j, 1.4013822078704834
loss:  0.0092+0.0000j, 1.4140253067016602
loss:  0.0085+0.0000j, 1.4264130592346191
loss:  0.0080+0.0000j, 1.4392167329788208
loss:  0.0075+0.0000j, 1.4524592161178589
loss:  0.0071+0.0000j, 1.4657750129699707
loss:  0.0068+0.0000j, 1.4790937900543213
loss:  0.0064-0.0000j, 1.4916974306106567
loss:  0.0061-0.0000j, 1.5038169622421265
loss:  0.0058+0.0000j, 1.515364408493042
loss:  0.0056+0.0000j, 1.5264441967010498
loss:  0.0053+0.0000j, 1.5372774600982666
loss:  0.0051+0.0000j, 1.547993063926

loss:  0.0003-0.0000j, 1.8816407918930054
loss:  0.0003-0.0000j, 1.8820574283599854
loss:  0.0003-0.0000j, 1.882530689239502
loss:  0.0003-0.0000j, 1.8830106258392334
loss:  0.0003+0.0000j, 1.883467197418213
loss:  0.0003-0.0000j, 1.8839054107666016
loss:  0.0003-0.0000j, 1.8840510845184326
loss:  0.0003-0.0000j, 1.884171485900879
loss:  0.0003-0.0000j, 1.8843109607696533
loss:  0.0003-0.0000j, 1.8845058679580688
loss:  0.0003-0.0000j, 1.8847758769989014
loss:  0.0003-0.0000j, 1.8850957155227661
loss:  0.0003-0.0000j, 1.8854484558105469
loss:  0.0003+0.0000j, 1.8858273029327393
loss:  0.0003-0.0000j, 1.8861910104751587
loss:  0.0003-0.0000j, 1.8865411281585693
loss:  0.0003-0.0000j, 1.8868656158447266
loss:  0.0003-0.0000j, 1.8871735334396362
loss:  0.0003+0.0000j, 1.8874690532684326
loss:  0.0003-0.0000j, 1.8877735137939453
loss:  0.0003-0.0000j, 1.888071060180664
loss:  0.0003-0.0000j, 1.8883886337280273
loss:  0.0003-0.0000j, 1.888731837272644
loss:  0.0003-0.0000j, 1.88904476165771

KeyboardInterrupt: 